<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width = 400, align = "center"></a>

<h1 align=center><font size = 5>FILTRO BASADO EN CONTENIDO</font></h1>

Los sistemas recomendadores son colecciones de algoritmos utilizados para sugerir elementos a los usuarios basados en información del usuario. Estos sistemas suelen verse en tiendas online, base datos de películas y buscadores de trabajos. En este labo, se explorarán Content-based sistemas recomendadores basados en contenidos e implementará una simple versión de uno utilizando Python las librerías Pandas.

### Table of contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
- <p><a href="#ref1">Obteniendo los Datos</a></p>
- <p><a href="#ref2">Preprocesamiento</a></p>
- <p><a href="#ref3">Filtro Basado en Contenido</a></p>
<p></p>
</div>
<br>

<a id="ref1"></a>
# Obteniendo los Datos

Para obtener y extraer los datos, solo corre los siguientes scripts Bash:  
Set de datos obtenidos en [GroupLens](http://grouplens.org/datasets/movielens/). Descargemos el set de datos. Para descargar los datos, utilizaremos **`!wget`**. Para descargar los datos, usaremos `!wget` que lo descargaremos del IBM Object Storage.  
__¿Sabías?__ Cuando se trata de Machine Learning, seguro trabajarás con grandes datasets (juego de datos). Entonces, ¿dónde podrás guardar esos datos? IBM ofrece una oportunidad única para las empresas, con 10 Tb de IBM Cloud Object Storage: [Registrate ahora gratuitamente](http://cocl.us/ML0101EN-IBM-Offer-CC)

In [ ]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('descomprimiendo ...')
!unzip -o -j moviedataset.zip 

¡Ahora estás listo para comenzar a trabajar con los datos!

<a id="ref2"></a>
# Preprocesamiento

Primero, saquemos a todos los imports del camino:

In [ ]:
#Librería de manipulación del marco de datos
import pandas as pd
#Funciones matemáticas, necesitaremos la función sqrt para importar sólo lo necesario
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Ahora carguemos cada archivo en su propio marco de datos:

In [ ]:
#Guardando la información de las películas dentro del marco de datos pandas
movies_df = pd.read_csv('movies.csv')
#Guardando la información del usuario dentro del marco de datos pandas
ratings_df = pd.read_csv('ratings.csv')
#Head es una función que obtiene los primeros N registros de un marco de datos. El valor por omision de N es 5.
movies_df.head()

Ahora saquemos el año de la columna __title__ utilizando la función replace y la guardamos en una nueva columna llamada __year__.

In [ ]:
#Utilizando expresiones regulares para encontrar un año guardado entre paréntesis
#Especificamos los paréntesis para no tener conflicto con las películas que tienen años como parte de su título
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Eliminando los paréntesis
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Eliminando los años de la columna 'title'
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Aplicando la función strip para eliminar los caracteres blancos finales
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

Con ello, separemos los valores de la columna __Genres__ y pongámoslo todos en __list of Genres__ para simplificar una utilización que haremos después. Esto también se puede lograr la función split string de Python dentro de la columna que corresponde.

In [ ]:
#Cada género está separado por un | para simplificar la llamada que se haga solo a |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

Ya que poner los géneros en una lista no es lo más óptimo para la técnica de sistemas recomendadores basados en contenidos, usaremos la técnica One Hot Encoding para convertir la lista de géneros en un vector donde cada colunna corresponde a un valor de la característica mencionada. Esta codificación se necesita para alimentar datos categóricos. En este caso, guardamos cada género distinto en colunnas que tienen 1 ó 0. 1 quiere decir que una película tiene género y 0 que no lo tiene. Guardames también un marco de datos en otra variable ya que los géneros no serán importante en esta primera instancia de sistemas recomendadores.

In [ ]:
#Copiando el marco de datos de la pelicula en uno nuevo ya que no necesitamos la información del género por ahora.
moviesWithGenres_df = movies_df.copy()

#Para cada fila del marco de datos, iterar la lista de géneros y colocar un 1 en la columna que corresponda
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
#Completar los valores NaN con 0 para mostrar que una película no tiene el género de la columna
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

A continuación, miremos los ratings de los marcos de datos.

In [ ]:
ratings_df.head()

Cada fila dentro de los ratings tienen un user id asociado con al menos una película, un rating y una marca de tiempo que muestra cuándo se revisó la película. No se necesitará la columna de la marca de tiempo , por lo que la podemos eliminar para ahorrar espacio de memoria.

In [ ]:
#La sentencia Drop elimina la fila o columna señalada del marco de datos
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

<a id="ref3"></a>
# Sistema de recomendación Basado en Contenido

Ahora miremos cómo implementar __Content-Based__ o __Item-Item recommendation systems__. Esta técnica intenta descubrir los aspectos favoritos del usuario para algún elemento, y luego recomienda elementos que presentan esos aspectos. En nuestro caso, vamos a intentar descubrir los géneros favoritos a partir de las películas y los ratings.

Comencemos por crear una entrada para el usuario para que recomiende películas

Nota: Para agregar más películas, aumenta la cantidad de elementos en userInput. Agrega tantos como desees! Solo asegúrate de escribir en letras mayúsculas y si una película comienza con un "The", como "The Matrix" entonces escríbelo así: 'Matrix, The' .

In [ ]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

#### Agregar movieId al ingreso del usuario
Con las datos ingresados completos, extraigamos los ID de las películas del dataframe de películas y agreguémosla.

Esto se logra primero sacando las filas que tienen que tienen títulos de películas y luego une este subconjunto con el dataframe de entrada. También sacamos columnas que no se necesitan para ahorrar espacio de memoria.

In [ ]:
#Filtrar las películas por título
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Luego juntarlas para obtener el movieId. Implícitamente, lo está uniendo por título.
inputMovies = pd.merge(inputId, inputMovies)
#Eliminando información que no utilizaremos del dataframe de entrada
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
#Dataframe de entrada final
#Si una película que se agregó no se encuentra, entonces podría no estar en el dataframe 
#original o podría estar escrito de otra forma, por favor revisar mayúscula o minúscula.
inputMovies

Ahora vamos a comenzar aprendiendo las preferencias del ingreso de datos, por lo que obtendremos el subconjunto de películas que se vieron a partir del marco de datos que contienen los géneros definidos con valores binarios.

In [ ]:
#Descartando las películas de la entrada de datos
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

Necesitaremos solamente la tabla actual de géneros, por lo que ordenaremos un poco inicializando el índice y eliminando las columnas movieId, title, genres e year.

In [ ]:
#Inicializando el índice para evitar problemas a futuro
userMovies = userMovies.reset_index(drop=True)
#Eliminando problemas innecesarios para ahorrar memoria y evitar conflictos
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

Ahora si estamos listos para comenzar a aprender las preferencias recibidas!

Para lograrlo, ponderaremos cada género. Esto se puede lograr utilizando las revisiones y multiplicádolas dentro de la tabla de ingreso de género para luego juntar la tabla resultante por columna. Esta operación en realidad es un producto escalar entre una matriz y un vector. Esto se logra invocando la función de Panda llamada "dot".

In [ ]:
inputMovies['rating']

In [ ]:
#Producto escalar para obtener los pesos
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#Perfil del usuario
userProfile

Ahora, tenemos los pesos para cada preferencia del usuario. Esto se conoce como Perfil del Usuario. Utilizando esto, podemos sugerir películas que satisfagan las preferencias del usuario.

Comencemos extrayendo la tabla de géner del marco de datos original:

In [ ]:
#Ahora llevemos los géneros de cada película al marco de datos original
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#Y eliminemos información innecesaria
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

In [ ]:
genreTable.shape

Habiendo completado el perfil de entrada y la lista completa de películas con sus respectivos géneros, llevaremos el peso promedio de cada película basado en el perfil de ingreso para luego recomendar las primeras veinte películas que más se adecuan a tal perfil.

In [ ]:
#Multiplicando los géneros por los pesos para luego calcular el peso promedio
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

In [ ]:
#Ordena nuestra recomendación en orden descendente
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Miremos los valores
recommendationTable_df.head()

Ahora vemos la tabla de recomendación!

In [ ]:
#Tabla de recomendaciones final
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

### Ventajas y Desventajas del Filtrado Basado en Contenido

##### Ventajas
* Se aprende de las preferencias del usuario
* Es muy personalizado para el usuario

##### Desventajas
* No se tiene en cuenta lo que otros piensan del item, por lo que recomendaciones de item de baja calidad podría suceder
* Extracting data is not always intuitive
* Determining what characteristics of the item the user dislikes or likes is not always obvious

## ¿Deseas aprender más?

IBM SPSS Modeler es una plataforma para analytics que contiene varios algoritmos de machine learning. Fue diseñada para acercar inteligencia predictiva a las decisiones hechas por individuos, grupos, sistemas, toda la empresa. Un free trial está disponible a través de este curso en: [SPSS Modeler](http://cocl.us/ML0101EN-SPSSModeler).

Asi mismo, puedes utilizar Watson Studio para ejecutar estos notebooks más rápido y con datasets más grandes. Watson Studio es una solución en la nube lider de IBM's para científicos de datos, construída por científicos de datos. Con Jupyter notebooks, RStudio, Apache Spark y librerías conocidas pre instaladas en la nube, Watson Studio posibilita a los científicos de datos colaborar en sus proyectos sin tener que instalar nada. Sumate a la comunidad de usuarios Watson Studio hoy mismo por medio de una cuenta gratuita en [Watson Studio](https://cocl.us/ML0101EN_DSX)

### ¡Gracias por completar la lección!

Notebook creado por: <a href = "https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, Gabriel Garcez Barros Sousa

<hr>
Copyright &copy; 2018 [Cognitive Class](https://cocl.us/DX0108EN_CC). Este lab y su código fuente fueron registrados bajo los términos de [MIT License](https://bigdatauniversity.com/mit-license/).​